In [ ]:
# Thanks for Code from https://www.kaggle.com/finlay/unsupervised-image-text-baseline-in-20min

In [1]:
DATA_PATH = '../../../Documents/shopee-product-matching/'
DEVICE = 'cuda'
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.image import imread
from IPython.display import Image
from PIL import Image

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

train = pd.read_csv(DATA_PATH + 'train.csv')

sample = train
img_phash = sample['image_phash']  # Series types
sample.head(5)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [2]:
converted_phash = []
for x in img_phash:
    temp = list(x)
    temp = [int(x, 16) for x in temp]
    converted_phash.append(list(temp))

In [3]:
from sklearn.preprocessing import StandardScaler
x = img_phash

converted_phash = []
for x in img_phash:
    temp = list(x)
    temp = [int(x, 16) for x in temp]
    converted_phash.append(list(temp))
    
converted_phash = StandardScaler().fit_transform(converted_phash)
converted_phash

features = range(0,16)
pd.DataFrame(converted_phash, columns=features).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-1.442896,-1.076365,0.183342,-0.004474,-1.266186,1.869770,0.404440,-0.962617,-0.295380,1.264776,-0.721575,1.056042,-1.420101,-0.695085,-1.013772,-0.913540
1,-0.961265,1.347040,-1.248367,1.756996,0.021773,-0.533956,-0.303997,-1.626159,0.924801,-1.147135,0.223726,-0.950572,0.027788,1.760292,-1.728181,1.766017
2,-0.479634,0.025183,-1.009749,1.096444,0.536957,-1.408038,-1.720870,1.470370,1.168837,-0.927870,1.641676,-0.504658,-1.902731,1.760292,-0.061227,0.202942
3,-1.924527,-0.856055,-1.725604,-0.665026,1.567324,1.214208,-0.540143,0.143286,1.412873,0.606982,1.878001,1.501956,0.510418,-1.141517,0.176909,-0.913540
4,-0.961265,-0.635746,1.615052,-0.885209,-2.038962,0.558647,1.821313,0.364467,-1.515560,-0.708605,0.696376,1.278999,-0.213527,-1.587949,0.176909,1.096128


In [4]:
from sklearn.decomposition import PCA
n = 16 # the number of principal components
pca = PCA(n_components = n) 
principalComponents = pca.fit_transform(converted_phash)
data_feature = principalComponents
principalDataFrame = pd.DataFrame(data=principalComponents, columns = range(0,n))

# to figure out the ratio of explained variance 
print('explained ratio : %.2f'% sum(pca.explained_variance_ratio_))


explained ratio : 1.00


In [5]:
data_feature = torch.tensor(principalDataFrame[:].values)
data_feature

tensor([[ 2.3705, -0.0284,  1.0440,  ...,  0.7568, -0.8804,  0.6838],
        [-0.8128,  2.8936, -0.0601,  ...,  0.0386, -0.5433, -0.0681],
        [-1.0122, -0.2543, -2.6544,  ...,  0.3464,  1.2043, -0.0719],
        ...,
        [-1.4149,  2.1801, -1.0070,  ...,  0.2119, -0.5701, -0.1760],
        [ 1.3595,  0.1988,  2.7947,  ...,  1.4761, -0.1224,  0.5389],
        [-1.7761,  2.2251,  0.4168,  ...,  0.0037,  0.3749, -0.3951]],
       dtype=torch.float64)

In [6]:
from sklearn.preprocessing import normalize

data_feature = np.vstack(data_feature)
data_feature = normalize(data_feature)

data_feature = torch.from_numpy(data_feature)
data_feature = data_feature.cuda()



In [7]:
preds = []
CHUNK = 1024*4

print('Finding similar images...')
CTS = len(data_feature)//CHUNK
if len(data_feature)%CHUNK != 0:
    CTS += 1
    
for j in range(CTS):
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b, len(data_feature))
    print('chunk', a, 'to', b)
    
    distances = torch.matmul(data_feature, data_feature[a:b].T).T
    distances = distances.data.cpu().numpy()
    
    #print(type(distances))
    #print(distances.shape)
    
    for k in range(b-a):
        IDX = np.where((distances[k,]>0.95)&(distances[k,]!=1.0))[0][:]
        o = sample.iloc[IDX].label_group.values
        
        preds.append(o)
        
sample['predicted_label'] = preds


Finding similar images...
chunk 0 to 4096
chunk 4096 to 8192
chunk 8192 to 12288
chunk 12288 to 16384
chunk 16384 to 20480
chunk 20480 to 24576
chunk 24576 to 28672
chunk 28672 to 32768
chunk 32768 to 34250


In [8]:
sample.head(10)

,posting_id,image,image_phash,title,label_group,predicted_label
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,[249114794]
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,[2937985045]
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,[2395904891]
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,[4093212188]
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,[]
5,train_2464356923,0013e7355ffc5ff8fb1ccad3e42d92fe.jpg,bbd097a7870f4a50,CELANA WANITA (BB 45-84 KG)Harem wanita (bisa...,2660605217,"[2660605217, 2660605217]"
6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137,[1835033137]
7,train_1806152124,0014f61389cbaa687a58e38a97b6383d.jpg,eea7e1c0c04da33d,KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT...,1565741687,[1565741687]
8,train_86570404,0019a3c6755a194cb2e2c12bfc63972e.jpg,ea9af4f483249972,"[LOGU] Tempelan kulkas magnet angka, tempelan ...",2359912463,[2359912463]
9,train_831680791,001be52b2beec40ddc1d2d7fc7a68f08.jpg,e1ce953d1a70618f,BIG SALE SEPATU PANTOFEL KULIT KEREN KERJA KAN...,2630990665,[]
